### Using Cohere's Generative model to extract topic entities  and score perfomance of the model.

Using named-entity recognition(NER) we extract information that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc. 

![NER Diagram](C:\Users\RYZEN\Cohere\Customer-Feedback-Analysis\NER.png)

This notebook also demonstrates the broader usecase of structured generation based on providing multiple examples in the prompt.


The data are job descriptions ( together named entities)  and  relationships between entities in json forma. If the model is unable to detect the entities, it will return "none".

## Setup
Let's start by installing the packages we need.

In [ ]:
!pip install cohere requests tqdm

We'll then import these packages and declare the function that retrieves post titles from reddit.

In [ ]:
#importing packages and reading data for training(fine tuning) the model
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

Get API key from cohere.ai

In [ ]:
# Paste your API key here. 
api_key = 'nidVfcm5xYEyW8etrREYrKmXjcdD65sCGVhxRJOl'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

## Training our prompt

In our prompt, we'll present the model with  news item dataset (e.g. title +  description + body) and the expected output we expect.

In [ ]:
import os
os.listdir()

In [ ]:
train = pd.read_json('../data/relations_dev_train.json')
test= pd.read_json('../data/relations_test.json')


In [33]:
from cProfile import label


def preprocess_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [34]:
train['label'] = train['tokens'].apply(preprocess_tokens)
test['label'] = test['tokens'].apply(preprocess_tokens)


In [35]:

job_entities = [
    ("Bachelor", "Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience"),
    ("BS/MS", "10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experience building and leading a software organization through product design, delivery and commercialization of consumer electronics devices. Experience recruiting and managing technical teams, including performance management. BS/MS in Computer Science. Experience in leading timeline, multi-partner initiatives. Organizational communication and coordination experience. PREFERRED 5+ years of experience with hands-on technical management, release engineering, tools engineering, DevOps, or related area."),
    ("BS", "BS degree in Computer Science or related field. 7+ years C++ experience, including C++11 features and principles. 5+ years experience creating software for real-time environments such as games or robotics. 2+ years experience managing software engineers. Proven track record of software development, including shipping one or more products on large code bases that span platforms and tools. Problem solving and optimization experience. Communication experience and demonstrated experience working across disciplines and teams to drive solutions. PREFERRED Hands-on experience with 3D computer vision algorithms including Calibration, SLAM, Reconstruction, Mapping, Localization, Sensor Fusion, State Estimation and Image Processing Experience with designing (products or open-source/git software) of inertial/optical sensing devices Publication in leading workshops or conferences such as CVPR, ECCV, ICCV, SIGGRAPH, ICCP, RSS, ICRA, etc."),
    ("BS/MS", "Experience in backend services, systems, and data infrastructure. BS/MS in Computer Science or a related technical field. 5+ years of work experience architecting, developing, and shipping software products and/or services. 4+ years of experience managing cross-functional engineering teams."),
    ("BS", "BS in Electrical Engineering or Computer Engineering, or related Engineering Degree, with 3+ years of industry experience. 3+ years of experience with PCB design with both integrated and discrete circuits, and high-speed interfaces. Experience with development and bring-up of data center or Enterprise infrastructure system equipment. Experience with analyzing and resolving system hardware issues. PREFERRED MS in electrical engineering or computer engineering. 5+ years of industry experience. Familiar with storage server architectures with HDD. Experience with end-to-end enabling, design, and deployment cycle of a product. Experience with working with thermal, mechanical, management firmware, system firmware, and software teams during product development. Experience with design and troubleshooting data buses used in server such as PCIe, SAS, SATA, I2C. Experience with industry standard EDA tools for developing PCB designs. Knowledge of using Linux for testing."),
    ("MS", "BS in Electrical Engineering or Computer Engineering, or related Engineering Degree, with 3+ years of industry experience. 3+ years of experience with PCB design with both integrated and discrete circuits, and high-speed interfaces. Experience with development and bring-up of data center or Enterprise infrastructure system equipment. Experience with analyzing and resolving system hardware issues. PREFERRED MS in electrical engineering or computer engineering. 5+ years of industry experience. Familiar with storage server architectures with HDD. Experience with end-to-end enabling, design, and deployment cycle of a product. Experience with working with thermal, mechanical, management firmware, system firmware, and software teams during product development. Experience with design and troubleshooting data buses used in server such as PCIe, SAS, SATA, I2C. Experience with industry standard EDA tools for developing PCB designs. Knowledge of using Linux for testing."),
    ("Bachelor", "Bachelor's Degree in Computing Science or a related subject Java experience Unix experience English: Fluent Preferred Tech and Prof Experience N/A EO Statement IBM is committed to creating a diverse environment and is proud to be an equal opportunity employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, gender, gender identity or expression, sexual orientation, national origin, genetics, disability, age, or veteran status. IBM is also committed to compliance with all fair employment practices regarding citizenship and immigration status. ."),
    ("BA/BS", "5+ years of software engineering work experience, including hands-on technical management.5+ years coding in one of the following languages: C++, Java, and/or PHP.Demonstrated ability to recruit and manage technical teams, including performance management.BA/BS in Computer Science (In lieu of degree, 4+ years relevant work experience)"),
    ("BS/MS", "Experience being technical, hands-on and a people manager for technical teams Experience leading, technically-challenging, cross-functional infrastructure projects from conception to end Experience with Unix internals (especially FreeBSD and Linux) Experience with multi-node cluster handling live traffic Knowledge of TCP/IP, experience with network programming Experience building and scaling distributed available systems 8+ years of work experience in software engineering, including 3+ years of proven hands-on technical management experience of software engineers BS/MS in Computer Science (In lieu of degree, work experience)"),
    ("BS/MS", "Basic : A BS/MS in Computer Science or related field Preferred : 5+ years of programming experience writing code in Java, C++, C#, or C or other object-oriented programming language Experience developing and testing computer software and/or online services Strong coding, debugging and problem-solving skills Strong knowledge of object-oriented programming language paradigms Great communication skills to collaborate cross-group and work effectively within the team Expertise in web or mobile application development Expertise in relational databases, distributed systems and/or big data technologies Experience in developing large scale services Ability to meet Microsoft, customer and/or government security screening requirements are required for this role. These requirements include, but are not limited to the following specialized security screenings: Microsoft Cloud Background Check: This position will be required to pass the Microsoft Cloud background check upon hire/transfer and every two years thereafter."),
    ("BA/BS degree", "Skills & : Exceptional management skills, with a proven ability to develop collaborative teams of exceptional engineers Knowledge of the Web UX industry today Expert understanding of web services and site technologies (for example, REST APIs , HTML, CSS, JavaScript, JSON etc.) Experience with newer technologies like AngularJS, Cordova, and ReactJS a plus Experience with scrum, Kanban, or other agile engineering techniques Experience Required: BA/BS degree in CS, related discipline or equivalent work experience 3 or more years of engineering management experience 4 or more years of Web UX design and coding experience in high scale services 7 or more years of general design and coding experience (C++, C#, and/or Java) //epicMEE Come join us and help shape how we do amazing customer experiences on the web!"),
    ("Bachelor", "include: Bachelor of Mechanical Engineering or equivalent. At least 5 years of HW engineering experience. Excellent communication, teamwork, documentation and organizational skills. Great attitude and self-motivation. Systematic and conceptual thinking. Proficiency with Mechanical design tool, like CREO or other. Comfortable in a lab environment using test and measurement equipment to characterize and analyze designs. Proficiency with board and system level design, analysis, verification, and documentation. Proficiency with Statistical analysis methods and tools such as JMP, preferred. Experience delivering solutions for high-mix high-volume production, preferred. Ability to troubleshoot to root cause and solve complex problems. SPC and six sigma methodologies."),
    ("BS/MS", "A successful member of our team will have: 5 + years of industry experience developing and implementing tools and applications at scale. BS/MS in Computer Science or equivalent industry experience. Demonstrated CS fundamentals with a deep appreciation for the differences between platforms and environments. Demonstrated technical design, problem solving and debugging skills. Unwavering passion to understand and simplify the developer experience. Fantastic communication skills. A track record of working collaboratively to understand customer needs and create solutions that positively impact our customers.")
]
# job_entities = [
#     ("text", "Bachelor",
#         "start", '0',
#         "end", '8',
#         "token_start", '0',
#         "token_end", '0',
#         "entityLabel", "DIPLOMA"),
#      ("text", "Mechanical Engineering",
#         "start", '21',
#         "end", '43',
#         "token_start", '4',
#         "token_end", '5',
#         "entityLabel", "DIPLOMA_MAJOR"),
#      ("text", "developing",
#         "start", '89',
#         "end", '99',
#         "token_start", '15',
#         "token_end", '15',
#         "entityLabel", "SKILLS"),
#      ("text", "connector related products",
#         "start", '137',
#         "end", '163',
#         "token_start", '22',
#         "token_end", '24',
#         "entityLabel", "SKILLS")
#  ]
job_entities

[('Bachelor',
  "Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience"),
 ('BS/MS',
  '10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experience building and leading a software organization through product design, delivery and commercialization of consumer electronics devices. Experi

Creating a prompt that demonstrates the task to the model. The prompt contains the examples above, and then presents the input text and asks the model to extract the movie name.


![Extraction prompt containing the examples and the input text](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/extraction-prompt-example.png)


In [36]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


cohereJobExtractor = cohereExtractor([e[1] for e in job_entities], 
                                       [e[0] for e in job_entities], [],
                                       "", 
                                       "extract diploma from the document:")


# cohereJobExtractor = cohereExtractor([e[1] for e in job_entities], 
#                                      [e[0] for e in job_entities], [],
#                                        "", 
#                                        "extract the EntityLabel:")

# Uncomment to inspect the full prompt:
# print(cohereMovieExtractor.make_prompt('<input text here>'))

In [37]:
# This is what the prompt looks like:
print(cohereJobExtractor.make_prompt('<input document here>'))

Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience
extract diploma from the document:Bachelor
---
10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experience building and leading a software organization through product design, delivery and commercialization of consumer electronics de

## Getting the data
Let's now make the API call to get the job list entities


In [ ]:
# #num_posts = 5

 # job_list = [
# #     "DIPLOMA",
# #     "DIPLOMA_MAJOR",
# #     "EXPERIENCE",
# #     "SKILLS"
# # ]

# job_list = [
#    ("Bachelor of Mechanical Engineering",
#     "At least 5 years of HW engineering experience",
#     "technical design, problem solving and debugging skills"),
#     ()
#     ]

# # Show the list


## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [41]:
test=pd.read_json('../data/relations_test.json')
results = []
for document in tqdm(test):
    try:
        extracted_text = cohereJobExtractor.extract('diploma')
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

 27%|██▋       | 3/11 [00:05<00:14,  1.86s/it]


Let's look at the results:

In [ ]:
pd.DataFrame(data={'text': df, 'extracted_text': results})

Looking at these results, the model got 9/10 correctly. It didn't pick up on Shaolin Soccer and God of Gambler in example \#4. It also called the second example "Pixar's Luca" instead of "Luca". But maybe we'll let this one slide.

When experimenting with extrction prompts, we'll often find edge-cases along the way. What if a post has two movies mentioned, for example? The more we run into such examples, the more examples we can add to the prompt that address these cases.

## How well does this work?
We can better measure the performance of this extraction method using a larger labeled dataset. So let's load a test set of 100 examples:

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

Let's look at some results:

In [ ]:
test_df.head()

Let's calculate the accuracy by comparing to the labeled examples

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

So it seems this prompt works well on this small test set. It's not guaranteed it will do as well on other sets, however. The prompt can be improved by trying on more data, discovering edge cases, and adding more examples to the prompt.

We can look at the examples it got wrong:

In [ ]:
test_df[test_df['correct']==0]

It indeed failed to pick up a few examples. Sometimes this uncovers edge cases and understandable mistakes (e.g. two films are mentioned in the text). 


We can look at the classification report for a more detailed look at what's included in the test set, and what the model got right and wrong:

In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print(classification_report(test_df['label'].str.lower(), test_df['extracted_text'].str.lower()))

This type of extraction is interesting because it doesn't just blindly look at the text. The model has picked up on movie information during its pretraining process and that helps it understand the task from only a few examples.

You can think about extending this to other subreddits, to extract other kinds of entities and information. [Let us know in the forum](https://community.cohere.ai/) what you experiment with and what kinds of results you see!

Happy building!